In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import sklearn
import os
%matplotlib inline
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, LSTM, Bidirectional
import tensorflow as tf

In [2]:
df = pd.read_csv("dataset_v1.csv")

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,6,7,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df[df['severity'].isin(["medium", "small", "large", "very_large", "na"])]
df['severity'].value_counts()
df = shuffle(df)
df.reset_index(inplace=True, drop=True)

In [4]:
def getX():
    X = df.copy()

    X.drop(X.columns[[i for i in range(0, 151)]], axis = 1, inplace = True)
    # X.drop(X.columns[[i for i in range(20, 35)]], axis = 1, inplace = True)
    X["severity"] = df["severity"]
    X.drop(X.columns[[0]], axis = 1, inplace = True)

    X = X.dropna()
    for i in range(0, 8):
        del X['air' + str(i)]
        del X['temp' + str(i)]
        del X['humidity' + str(i)]
        
    X.drop(X.columns[[i for i in range(len(X.columns)-8, len(X.columns)-2)]], axis = 1, inplace = True)
    return X
X = getX()

In [5]:
def generate_labels(binary = False):
    global X
    X = getX()
    y = []
    idx_to_severity = ["na", "small", "medium", "large", "very_large"]
    for severity in X.severity:
        if not binary:
            y.append(idx_to_severity.index(severity))
        elif severity == "na":
            y.append(0)
        else:
            y.append(1)
    X.drop(X.columns[[-1,-2]], axis = 1, inplace = True)
    print(y.count(1))
    return y

In [6]:
binary = True
y = generate_labels(binary)
X.head()

4394


,precip7,wind7,precip6,wind6,precip5,wind5,precip4,wind4,precip3,wind3
0,0.1,32,0.1,33,0.1,27,0.0,23,0.1,31
1,0.9,17,3.4,14,0.0,9,7.5,9,3.1,8
2,3.3,20,3.0,18,3.8,20,2.8,18,6.9,13
3,0.3,9,0.2,11,1.2,8,0.0,9,0.0,14
4,0.2,27,0.0,29,4.5,30,0.1,32,5.7,32


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train[0]

array([ 2.73977313,  4.43049107,  0.46606758,  4.50900482,  0.54848729,
        3.45024698,  0.06649168,  2.76014383, -0.14548434,  2.3531288 ])

In [8]:
y_train = np.array(y_train)
y_test = np.array(y_test)
# len(X_train) # of batches
# 5 timesteps (days 7 - 3)
# 2 features per timestep
X_train=X_train.reshape((len(X_train), 5, 2))
X_test=X_test.reshape((len(X_test), 5, 2))
print(X_train.shape)
X_train[0]

(7096, 5, 2)


array([[ 2.73977313,  4.43049107],
       [ 0.46606758,  4.50900482],
       [ 0.54848729,  3.45024698],
       [ 0.06649168,  2.76014383],
       [-0.14548434,  2.3531288 ]])

In [9]:
model = Sequential()
model.add(Bidirectional(LSTM(11)))
model.add(Dense(1))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(X_train, y_train, epochs = 20, batch_size = 20, verbose = 2)

Epoch 1/20
355/355 - 1s - loss: 1.2429 - accuracy: 0.4615
Epoch 2/20
355/355 - 1s - loss: 0.6914 - accuracy: 0.5390
Epoch 3/20
355/355 - 1s - loss: 0.6837 - accuracy: 0.5631
Epoch 4/20
355/355 - 1s - loss: 0.6800 - accuracy: 0.5695
Epoch 5/20
355/355 - 1s - loss: 0.6777 - accuracy: 0.5757
Epoch 6/20
355/355 - 1s - loss: 0.6769 - accuracy: 0.5757
Epoch 7/20
355/355 - 1s - loss: 0.6760 - accuracy: 0.5812
Epoch 8/20
355/355 - 1s - loss: 0.6758 - accuracy: 0.5853
Epoch 9/20
355/355 - 1s - loss: 0.6756 - accuracy: 0.5795
Epoch 10/20
355/355 - 1s - loss: 0.6748 - accuracy: 0.5815
Epoch 11/20
355/355 - 1s - loss: 0.6752 - accuracy: 0.5831
Epoch 12/20
355/355 - 1s - loss: 0.6735 - accuracy: 0.5837
Epoch 13/20
355/355 - 1s - loss: 0.6740 - accuracy: 0.5855
Epoch 14/20
355/355 - 1s - loss: 0.6743 - accuracy: 0.5800
Epoch 15/20
355/355 - 1s - loss: 0.6743 - accuracy: 0.5836
Epoch 16/20
355/355 - 1s - loss: 0.6739 - accuracy: 0.5854
Epoch 17/20
355/355 - 1s - loss: 0.6729 - accuracy: 0.5848
Epoch 

In [10]:
model.evaluate(X_test, y_test)

56/56 [==============================] - 0s 1ms/step - loss: 0.6819 - accuracy: 0.5722


[0.6818662285804749, 0.5721533298492432]